In [20]:
import pandas as pd
# Importing the data
DATA_PATH_GENRE = 'https://raw.githubusercontent.com/Spotify-Song-Predict-DS18/Song-Predict/main/data/data_w_genres.csv'
DATA_PATH_GENRE_2 = 'https://raw.githubusercontent.com/Spotify-Song-Predict-DS18/Song-Predict/main/data/data_by_genres.csv'
DATA_PATH_ARTIST = 'https://raw.githubusercontent.com/Spotify-Song-Predict-DS18/Song-Predict/main/data/data_by_artist.csv'
DATA_PATH = 'https://raw.githubusercontent.com/Spotify-Song-Predict-DS18/Song-Predict/main/data/data.csv'
genres = pd.read_csv(DATA_PATH_GENRE)
genre2 = pd.read_csv(DATA_PATH_GENRE_2)
artist = pd.read_csv(DATA_PATH_ARTIST)
control = pd.read_csv(DATA_PATH)

In [3]:
#CLEANING DATA: REMOVING '' and [] SYMBOLS

list_of_genres = []
for i in genres['genres']:
    i = i[1:-1].replace("'", "")
    list_of_genres.append(i)
    
genres.drop(['genres'], axis=1, inplace=True)
genres = genres.assign(New_genres=list_of_genres)
genres = genres.rename(columns={'New_genres':'genre', 'artists':'artist'})

#removing empty values from 'genre' column
genres = genres[genres['genre'] != '']

In [4]:
#CLEANING "ARTISTS": REMOVING [] and ''
list_of_artists  = []
for k in control['artists']:
    k = k[1:-1].replace("'", "")
    list_of_artists.append(k)
list_of_artists   
control.drop(['artists'], axis=1, inplace=True)
control = control.assign(New_artists=list_of_artists)
control = control.rename(columns={'New_artists':'artist'})

In [5]:
# Merging 
result = pd.merge(control, genres[['artist', 'genre']], how='left', on='artist')
result = result[result['genre'].isnull() == False]

In [6]:
# Transforming my dataset
# TODO - Dates need to be coherent
# TODO - might have to make it into a function. 

result['release_date'] = pd.to_datetime(result['release_date'])

In [21]:
# looking at the difference in popularity
result['popularity'].describe()

count    121551.000000
mean         36.233894
std          18.653777
min           0.000000
25%          24.000000
50%          38.000000
75%          50.000000
max         100.000000
Name: popularity, dtype: float64

In [7]:
# TODO - Build a recommender model with machine learning. 
from surprise import Reader, Dataset, SVD 
from surprise.model_selection.validation import cross_validate

# Intantiating my reader and my data
reader = Reader(rating_scale=(0,100))
data = Dataset.load_from_df(result[['artist', 'id', 'popularity']],
                           reader)
# Intatntiating my SVD 
svd = SVD()

In [35]:
# Running a 5-fold cross-validation 
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=100,
              verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 100 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Fold 11 Fold 12 Fold 13 Fold 14 Fold 15 Fold 16 Fold 17 Fold 18 Fold 19 Fold 20 Fold 21 Fold 22 Fold 23 Fold 24 Fold 25 Fold 26 Fold 27 Fold 28 Fold 29 Fold 30 Fold 31 Fold 32 Fold 33 Fold 34 Fold 35 Fold 36 Fold 37 Fold 38 Fold 39 Fold 40 Fold 41 Fold 42 Fold 43 Fold 44 Fold 45 Fold 46 Fold 47 Fold 48 Fold 49 Fold 50 Fold 51 Fold 52 Fold 53 Fold 54 Fold 55 Fold 56 Fold 57 Fold 58 Fold 59 Fold 60 Fold 61 Fold 62 Fold 63 Fold 64 Fold 65 Fold 66 Fold 67 Fold 68 Fold 69 Fold 70 Fold 71 Fold 72 Fold 73 Fold 74 Fold 75 Fold 76 Fold 77 Fold 78 Fold 79 Fold 80 Fold 81 Fold 82 Fold 83 Fold 84 Fold 85 Fold 86 Fold 87 Fold 88 Fold 89 Fold 90 Fold 91 Fold 92 Fold 93 Fold 94 Fold 95 Fold 96 Fold 97 Fold 98 Fold 99 Fold 100Mean    Std     
RMSE (testset)    13.2363 13.4268 13.2490 13.7291 13.3363 13.7595 12.8713 13.7995 13.3488 13.0554 13.5013 13.

{'test_rmse': array([13.23629501, 13.4267719 , 13.24897763, 13.72914828, 13.33634457,
        13.75950167, 12.87129678, 13.79952383, 13.34884608, 13.05544264,
        13.5012532 , 13.89226035, 13.12212664, 13.59321719, 13.58630537,
        13.26840412, 13.11247935, 13.28684237, 13.2091274 , 13.46513515,
        13.41416766, 13.13307367, 13.33411436, 13.43590021, 13.0638374 ,
        13.15335531, 13.33207006, 13.29532264, 13.20992929, 13.07839178,
        12.99007233, 13.20492632, 13.45843707, 12.82169659, 13.41318654,
        13.02376767, 13.23765845, 14.04142653, 13.1435739 , 13.31925144,
        13.05613583, 13.52855981, 13.16024403, 13.62194707, 13.34379684,
        13.48101661, 13.69233711, 13.17258126, 13.02963152, 13.64693532,
        13.40611108, 13.11532767, 13.36812343, 13.05860298, 13.43662883,
        13.28375494, 13.45056117, 13.41410704, 13.37589839, 13.03297798,
        13.10396876, 12.86979417, 13.3310104 , 13.59020486, 13.55482606,
        13.26727052, 13.87226339, 12.9

In [36]:
# Retraining the model using the entire dataset
trainset = data.build_full_trainset()
svd.fit(trainset)

In [37]:
# Model has been trained, time to use for prediction.
titles = result.copy()

titles['Estimate_Score'] = titles['id'].apply(lambda x: svd.predict('Circles', x).est)

In [38]:
titles = titles.sort_values(by=['Estimate_Score'], ascending=False)
titles.head(10)

,acousticness,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,...,name,popularity,release_date,speechiness,tempo,valence,year,artist,genre,Estimate_Score
126125,0.02170,0.755,202887,0.617,0,7na7Bk98usp84FaOJFPv3d,0.000000,9,0.0625,-5.144,...,Tattoo,89,2020-02-13,0.1820,193.896,0.745,2020,Rauw Alejandro,latin,38.772031
97637,0.17700,0.880,157606,0.751,0,2tnVG71enUj33Ic2nFN6kZ,0.000064,7,0.1060,-4.258,...,Ride It,90,2019-07-26,0.0874,117.948,0.884,2019,Regard,pop edm,38.750339
116709,0.12500,0.805,184255,0.562,0,4lMOQR1rRW5hJgJXOtKraL,0.000005,7,0.1480,-3.117,...,Relación,86,2020-04-02,0.1050,172.094,0.767,2020,Sech,"latin, panamanian pop, reggaeton",38.735951
87804,0.19400,0.729,183907,0.625,1,1xzBco0xcoJEDXktl7Jxrr,0.009860,4,0.2480,-5.266,...,Mo Bamba,80,2018-10-05,0.0315,146.034,0.261,2018,Sheck Wes,"rap, trap, underground hip hop, vapor trap",38.679869
97665,0.17700,0.866,223258,0.438,0,2W4yABdLc4Jnds67Poi5Nl,0.000000,7,0.0743,-8.943,...,Yellow Hearts,81,2019-06-18,0.0719,129.039,0.695,2019,Ant Saunders,pop,38.668999
169853,0.09460,0.697,190955,0.921,0,2ZEq4HT450Ye9IFGPTl9qV,0.000019,5,0.1590,-4.283,...,Lonely,81,2020-01-24,0.0424,123.988,0.722,2020,Joel Corry,"deep groove house, house, tropical house, uk d...",38.663566
87915,0.58700,0.572,201084,0.406,1,2nC3QhMI9reBIOWutbU3Tj,0.000004,10,0.1020,-8.624,...,Moral of the Story,84,2019-02-14,0.0427,119.812,0.265,2019,Ashe,indie pop,38.663420
152624,0.00181,0.760,248036,0.964,0,6ho0GyrWZN3mhi9zVRW7xi,0.703000,2,0.0975,-5.844,...,Losing It,78,2018-07-13,0.0577,125.003,0.643,2018,FISHER,"australian house, house",38.618978
107177,0.83100,0.641,148640,0.355,0,6qL5UWxRSQYml9O99ozoLv,0.000855,5,0.1100,-10.599,...,Love Songs - Bonus,77,2019-11-15,0.0906,120.032,0.453,2019,Kaash Paige,deep pop r&b,38.603116
85858,0.10300,0.622,220293,0.807,0,6x4tKaOzfNJpEJHySoiJcs,0.000000,5,0.3060,-6.752,...,Mambo No. 5 (a Little Bit of...),77,1999-07-19,0.4080,174.060,0.892,1999,Lou Bega,latin pop,38.591608
